In [1]:
// load up the spark context
/*
Spark Web UI available at http://172.17.0.2:4041
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1503805246563)
SparkSession available as 'spark'
*/

Intitializing Scala interpreter ...

Spark Web UI available at http://172.17.0.2:4041
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1503805246563)
SparkSession available as 'spark'


In [39]:
// Load some data into a dataset 
import org.apache.spark.sql.{ Dataset, DataFrame, SparkSession }
import org.apache.spark.sql.functions._
import org.apache.spark.sql._

case class Friends(name: String, friends: String) { 
    def toDelimitedString:String = s"$name|$friends"
}

case class Character(name: String, 
                     height: Integer, 
                     weight: Option[Integer], 
                     eyecolor: Option[String], 
                     haircolor: Option[String], 
                     jedi: String,
                     species: String) {
    def toDelimitedString:String = Seq(
        name, 
        height.toString,
        weight.map(_.toString).getOrElse(""), 
        eyecolor.getOrElse(""),
        haircolor.getOrElse(""),
        jedi, 
        species
    ).mkString("|")
}

// make sure this scope is visable everywhere
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

import org.apache.spark.sql.{Dataset, DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql._
defined class Friends
defined class Character


In [9]:
val friends: Dataset[Friends] = //
 spark
  .read
  .option("header", "true")
  .option("delimiter", "|")
  .option("inferSchema", "true")
  .csv("star.wars.friends.csv")
  .as[Friends]

friends.show()

+----------------+--------------------+
|            name|             friends|
+----------------+--------------------+
|            Yoda|      Obi-Wan Kenobi|
|Anakin Skywalker|     Sheev Palpatine|
|  Luke Skywalker|Han Solo, Leia Sk...|
|  Leia Skywalker|      Obi-Wan Kenobi|
| Sheev Palpatine|    Anakin Skywalker|
|        Han Solo|Leia Skywalker, L...|
|  Obi-Wan Kenobi|  Yoda, Qui-Gon Jinn|
|           R2-D2|               C-3PO|
|           C-3PO|               R2-D2|
|      Darth Maul|     Sheev Palpatine|
|       Chewbacca|            Han Solo|
|Lando Calrissian|            Han Solo|
|           Jabba|           Boba Fett|
+----------------+--------------------+



characterFriends: org.apache.spark.sql.Dataset[Friends] = [name: string, friends: string]


In [14]:
// Load character data into DataSet 
val characters: Dataset[Character] = //
 spark
  .read
  .option("header", "true")
  .option("delimiter", ";")
  .option("inferSchema", "true")
  .csv("star.wars.characters.csv")
  .as[Character]

characters.show()

+----------------+------+------+--------+---------+-------+-----------+
|            name|height|weight|eyecolor|haircolor|   jedi|    species|
+----------------+------+------+--------+---------+-------+-----------+
|Anakin Skywalker|   188|    84|    blue|    blond|   jedi|      human|
|   Padme Amidala|   165|    45|   brown|    brown|no_jedi|      human|
|  Luke Skywalker|   172|    77|    blue|    blond|   jedi|      human|
|  Leia Skywalker|   150|    49|   brown|    brown|no_jedi|      human|
|    Qui-Gon Jinn|   193|    89|    blue|    brown|   jedi|      human|
|  Obi-Wan Kenobi|   182|    77|bluegray|   auburn|   jedi|      human|
|        Han Solo|   180|    80|   brown|    brown|no_jedi|      human|
| Sheev Palpatine|   173|    75|    blue|      red|no_jedi|      human|
|           R2-D2|    96|    32|    null|     null|no_jedi|      droid|
|           C-3PO|   167|    75|    null|     null|no_jedi|      droid|
|            Yoda|    66|    17|   brown|    brown|   jedi|     

characters: org.apache.spark.sql.Dataset[Character] = [name: string, height: int ... 5 more fields]


In [47]:
val brownHair:Dataset[Character] = characters.filter{ x: Character => x.haircolor == "brown"}
// brownHair.show()
// println("Characters with brown hair -" + brownHair.count)
// save a file 
new java.io.PrintWriter("star.wars.brown.hair.csv") { s:Character => brownHair.map(_.toDelimitedString); close }

val blueEyes:Dataset[Character] = characters.filter{ x => x.eyecolor == "blue"}
// blueEyes.show()
// println("Characters with brown hair -" + blueEyes.count)
new java.io.PrintWriter("star.wars.brown.hair.csv") { s => blueEyes.map(_.toDelimitedString); close }


<console>: 54: error: overloaded method value filter with alternatives:

In [45]:
//
characters.filter("haircolor = brown")
     .join(friends, friends("name") === characters("name"))
     .groupBy("eyecolor", "weight")
     .agg(avg("eyecolor"), max("weight"))

<console>: 53: error: not found: value friends